# --- Day 7: Camel Cards ---
https://adventofcode.com/2023/day/7

## --- Part One ---

In [65]:
# some imports i may need
from collections import Counter
import functools


# init some vars
scores = []

# init a dict of card classifications
results = {'high_card':[],'one_pair':[],'two_pair':[],'three_kind':[],'full_house':[],'four_kind':[],'five_kind':[]}
# card value conversion list
card_value = {'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'T':10,'J':11,'Q':12,'K':13,'A':14}

# function to classify a hand
def classify_hand(hand):
    check = Counter(hand)
    # based on the length of unique items we can classifiy it
    match len(check):
        case 5:
            # always high card
            return 'high_card'
        case 4:
            # always one pair
            return 'one_pair'
        case 3:
            # two pair or three of a kind
            if check.most_common()[0][1] == 2:
                return 'two_pair'
            else:
                return 'three_kind'
        case 2:
            # either full house or 4 of a kind
            if check.most_common()[0][1] == 3:
                return 'full_house'
            else:
                return 'four_kind'
        case 1:
            return 'five_kind'
        case _:
            return ''   

    

with open("input.txt", 'r') as f:
# with open("sample.txt", 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        items = line.split(' ')
        hand = [card for card in items[0]]
        bid = int(items[1])
        out = [hand,bid]
        # classify the hand
        hand_class = classify_hand(hand)
        # append the [hand,bid] to the right result classification
        results[hand_class].append(out)        

# custom sort function that sorts based on card value
def compare_hands(hand1,hand2):
    h1 = hand1[0]
    h2 = hand2[0]
    for i in range(0,len(h1)):
        if card_value[h1[i]] > card_value[h2[i]]:
            return 1
        if card_value[h1[i]] < card_value[h2[i]]:
            return -1
    return 1

# init a initial rank
rank = 1
for key in results:
    # grab and sort results
    row = sorted(results[key],key=functools.cmp_to_key(compare_hands))
    # go through each hand and score it; append score to scores list; up rank by 1
    for hand in row:
        score = hand[1]*rank
        scores.append(score)
        rank +=1

# print sum of scores
print("\nThe answer to part 1 is:",sum(scores))


The answer to part 1 is: 251106089


## --- Part Two ---

In [83]:
# some imports i may need
from collections import Counter
import functools


# init some vars
scores = []

# init a dict of card classifications
results = {'high_card':[],'one_pair':[],'two_pair':[],'three_kind':[],'full_house':[],'four_kind':[],'five_kind':[]}
# card value conversion list; CHANGE: J=1 NOW
card_value = {'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'T':10,'J':1,'Q':12,'K':13,'A':14}

# function to classify a hand
def classify_hand(hand):
    check = Counter(hand)
    # get the amount of JOKERS
    js = check['J']
    # if there are 5 J's return five_kind else it breaks the loop below
    if js >= 5:
        return 'five_kind'
    # remove the J from the counter list
    del check['J']
    # find the most frequent other card
    frequent = check.most_common()[0][0]
    # increase that count by the amount of J's
    check[frequent] += js
    # based on the length of unique items we can classifiy it
    match len(check):
        case 5:
            # always high card
            return 'high_card'
        case 4:
            # always one pair
            return 'one_pair'
        case 3:
            # two pair or three of a kind
            if check.most_common()[0][1] == 2:
                return 'two_pair'
            else:
                return 'three_kind'
        case 2:
            # either full house or 4 of a kind
            if check.most_common()[0][1] == 3:
                return 'full_house'
            else:
                return 'four_kind'
        case 1:
            return 'five_kind'
        case _:
            return ''   

    

with open("input.txt", 'r') as f:
# with open("sample.txt", 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        items = line.split(' ')
        hand = [card for card in items[0]]
        bid = int(items[1])
        out = [hand,bid]
        # classify the hand
        hand_class = classify_hand(hand)
        # append the [hand,bid] to the right result classification
        results[hand_class].append(out)        

# custom sort function that sorts based on card value
def compare_hands(hand1,hand2):
    h1 = hand1[0]
    h2 = hand2[0]
    for i in range(0,len(h1)):
        if card_value[h1[i]] > card_value[h2[i]]:
            return 1
        if card_value[h1[i]] < card_value[h2[i]]:
            return -1
    return 1

# init a initial rank
rank = 1
for key in results:
    # grab and sort results
    row = sorted(results[key],key=functools.cmp_to_key(compare_hands))
    # go through each hand and score it; append score to scores list; up rank by 1
    for hand in row:
        score = hand[1]*rank
        scores.append(score)
        rank +=1

# print sum of scores
print("\nThe answer to part 2 is:",sum(scores))


The answer to part 2 is: 249620106
